In [1]:
import pandas as pd
import os
import json
import numpy as np
# import streamlit as st
# import streamlit.components.v1 as components
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
import sys
DIR = os.getenv('KEA_BASE_DIR')
sys.path.append(DIR)
import application
from application import *

In [2]:
df = get_vote_margin_data()

2023-09-09 10:31:49.047 
  command:

    streamlit run c:\Users\bhagy\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
C:\Users\bhagy\Documents\Jupyter Notebooks\Karnataka Election Analysis\application.py:137: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection) #, dtype=df_config


In [3]:
df.describe()

,votes,margin,year,margin percent
count,673.000000,673.000000,673.000000,673.000000
mean,76074.231798,19131.650817,2018.000000,41.007207
std,21595.737656,16085.976101,4.082483,51.015376
min,31737.000000,0.000000,2013.000000,0.000000
25%,61369.000000,7292.000000,2013.000000,11.580000
50%,75061.000000,15103.000000,2018.000000,26.810000
75%,86983.000000,26718.000000,2023.000000,52.690000
max,196220.000000,122392.000000,2023.000000,593.240000


In [36]:
fig = px.histogram(df[df.year==2023], x="margin percent",nbins=100)
fig.show()

In [43]:
fig = ff.create_distplot([list(df.loc[(df.year==2013)&(df['margin percent']<75.0),'margin percent']),list(df.loc[(df.year==2018)&(df['margin percent']<75.0),'margin percent']),list(df.loc[(df.year==2023)&(df['margin percent']<75.0),'margin percent'])],group_labels=['2013','2018','2023'],show_hist=False)
# fig.update_traces(meanline_visible=True)
fig = ff.create_distplot([list(df.loc[(df.year==2013),'margin']),list(df.loc[(df.year==2018),'margin']),list(df.loc[(df.year==2023),'margin'])],group_labels=['2013','2018','2023'],show_hist=False)
fig.show()

In [31]:
connection = create_connection()
query = r"""SELECT * FROM (WITH temp AS (SELECT name, t1.constituency, party, votes, year, const_num,
ROUND((votes/(SUM(votes) OVER(PARTITION BY year, t1.constituency))*100),2) AS `vote share percent`,
DENSE_RANK() OVER(PARTITION BY year, t1.constituency ORDER BY votes DESC) AS `rank`
FROM (SELECT name, constituency, party, votes, '2018' AS year FROM kea.candid18
UNION
SELECT name, constituency, party, votes, '2013' AS year FROM kea.candid13
UNION
SELECT name, constituency, party, votes, '2023' AS year FROM kea.candid23) t1
JOIN kea.const_num c ON t1.constituency = c.constituency) 
SELECT *, votes - LEAD(votes) OVER(PARTITION BY year, constituency ORDER BY votes DESC) AS `margin`,
`vote share percent` - LEAD(`vote share percent`) OVER(PARTITION BY year, constituency ORDER BY votes DESC) AS `margin percent`
FROM temp) temp2
WHERE `rank`=1"""

df = pd.read_sql_query(query, connection)

C:\Users\bhagy\AppData\Local\Temp\ipykernel_17280\834754634.py:16: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [32]:
fig = px.histogram(df[df.year=='2013'], x="margin percent",nbins=50)
fig.show()

In [35]:
df = df.dropna()
df.isnull().sum()

name                  0
constituency          0
party                 0
votes                 0
year                  0
const_num             0
vote share percent    0
rank                  0
margin                0
margin percent        0
dtype: int64

In [43]:
fig = ff.create_distplot([list(df.loc[(df.year=='2013'),'margin percent']),list(df.loc[(df.year=='2018'),'margin percent']),list(df.loc[(df.year=='2023'),'margin percent'])],group_labels=['2013','2018','2023'],show_hist=False)

In [44]:
fig.show()

In [38]:
df.loc[df['year']=='2013','margin percent'].describe()

count    223.000000
mean      14.813274
std       15.802979
min        0.090000
25%        4.515000
50%       10.770000
75%       19.415000
max       95.870000
Name: margin percent, dtype: float64

In [39]:
df.loc[df['year']=='2018','margin percent'].describe()

count    224.000000
mean      13.617500
std       14.865487
min        0.160000
25%        4.945000
50%        9.630000
75%       16.402500
max       93.640000
Name: margin percent, dtype: float64

In [40]:
df.loc[df['year']=='2023','margin percent'].describe()

count    224.000000
mean      12.575938
std       12.157323
min        0.020000
25%        4.650000
50%        9.270000
75%       17.255000
max       82.440000
Name: margin percent, dtype: float64

In [60]:
# Reading the GeoJSON
with open("AC_Boundary_simplified.json",'r',encoding='utf-8') as file:
    acmap = json.loads(file.read())

In [61]:
# This part of the JSON contains critical information
acmap['features'][0]['properties']

{'OBJECTID': 610,
 'AC_CODE': '1',
 'AC_NAME': 'Nippani',
 'PC_CODE': '01',
 'KGISDistri': '01',
 'AC_NAME_KN': 'ನಿಪ್ಪಾಣಿ',
 'Shape_STAr': 425991971.009,
 'Shape_STLe': 207237.810237}

In [62]:
# Fetch the constituency number and the corresponding constituency name from GeoJSON and convert to Dataframe

temp_dict = {}

for feature in acmap['features']:
    temp_dict.update({int(feature['properties']['AC_CODE']):feature['properties']['AC_NAME'].strip()})

In [63]:
# Fetch the constituency number and the corresponding constituency name from OpenCity dataset and convert to Dataframe

with open(f"{DIR}/2013 Elections\const_num.json",'r',encoding='utf-8') as file:
    const_num = json.loads(file.read())

In [66]:
# Correcting constituency names and numbers in the GeoJSON
for feature in acmap['features']:
    feature['properties']['AC_NAME'] = const_num['Constituency Name'][feature['properties']['AC_CODE'].lstrip('0')]
    feature['properties']['AC_CODE'] = feature['properties']['AC_CODE'].lstrip('0')

In [67]:
# Saving the JSON
with open("AC_Boundary_simplified.json",'w',encoding='utf-8') as file:
    file.write(json.dumps(acmap))

In [68]:
with open("BBMP.json",'r',encoding='utf-8') as file:
    bbmp = json.loads(file.read())

In [69]:
# Correcting constituency names and numbers in the GeoJSON
for feature in bbmp['features']:
    feature['properties']['AC_NAME'] = const_num['Constituency Name'][feature['properties']['AC_CODE'].lstrip('0')]
    feature['properties']['AC_CODE'] = feature['properties']['AC_CODE'].lstrip('0')

In [70]:
# Saving the JSON
with open("BBMP.json",'w',encoding='utf-8') as file:
    json.dump(bbmp, file)

In [71]:
# Define a scaling factor
scale_factor = 6.25

VERTICAL_OFFSET = 1.9
HORIZONTAL_OFFSET = 0.6

# Define center point of your map for scaling
# Update the code here to handle list of coordinates
num_coords = 0
total_lat = 0
total_lon = 0

for feature in bbmp['features']:
    for ring in feature['geometry']['coordinates']:
        for coord in ring:
            total_lat += coord[1]
            total_lon += coord[0]
            num_coords += 1

center_lat = total_lat / num_coords
center_lon = total_lon / num_coords

# Iterate over all coordinates and scale
for feature in bbmp['features']:
    for ring in feature['geometry']['coordinates']:
        for coord in ring:
            # Calculate offset from center point
            lat_offset = coord[1] - center_lat
            lon_offset = coord[0] - center_lon

            # Scale offset and recenter
            coord[1] = center_lat + (lat_offset * scale_factor) + VERTICAL_OFFSET
            coord[0] = center_lon + (lon_offset * scale_factor) + HORIZONTAL_OFFSET

# Save your scaled GeoJSON
with open('scaled_BBMP.json', 'w') as f:
    json.dump(bbmp, f)

In [72]:
with open('AC_Boundary_simplified.json', 'r') as f:
    acmap = json.load(f)

acmap['features'].extend(bbmp['features'])

# Save combined GeoJSON
with open('combined.json', 'w') as f:
    json.dump(acmap, f)